In [1]:
# Imports
import plotly.graph_objs as go
import plotly_express as px 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.subplots import make_subplots
from statsmodels.graphics.gofplots import qqplot
import scipy.stats as sps

import numpy as np
import pandas as pd
import scipy

init_notebook_mode(connected=True)

import matplotlib.pyplot as plt 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split


from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

There are certain set of assumptions that applicable when working with regression problems .Take for instance  linear regression where we have the following assumptions- 
1) we assume that we have a linear relationship between independent variable and the target variable. 
2) our data is homoscedastic  
3) residuals have a normal distributions 
4) Minimal multicollinearity 

The subject of this notebook is the third point- How do we know that the residuals due to linear regression model is normally distributed or not? This leads to a more general question. Given a dataset, can we say that the data is normally distributed or not? This seems like a fairly trivial question. Well, just plot the histogram of the data and see if it looks like a normal distribution. Histograms can be deceptive, it depends on the number of bins you choose and depends on the number of data points you have. 

Thankfully, there are certain tools available to us in order to determine if a dataset comes from a normal distribution or not. 

In this notebook we are going to cover the following tools- 
1) Graphical way: Histogram
2) Graphical way: Quantile-quantile(qq) plots 

In the next notebook we are going to look at the other two tests 
3) Statistical test: K-S test 
4) Statistical test: Anderson-Darling tests

The collection of tests that are used to determine if a data is normal are called normality tests.


Before we get into it, let us set up a problem. We generate a dataset and set up a linear regression problem. We fit a model to it and get the residues. 


In [2]:
# # generate data and visualize it
np.random.seed(1)
x = np.arange(0,1000)
noise = np.random.normal(0,10,1000)
slope = 0.1
b = 5.0 

y = (slope*x)+b 
y_noised =  y+noise



# test train split 
x_train, x_test, y_train, y_test = train_test_split(x,y_noised, test_size=0.2, random_state=1)

x_train_shape = x_train.shape[0]
y_train_shape = y_train.shape[0]

x_train_reshaped = x_train.reshape(x_train_shape, 1)
y_train_reshaped = y_train.reshape(y_train_shape, 1)


x_test_shape = x_test.shape[0]
x_test_reshaped = x_test.reshape(x_test_shape, 1)


# fitting the model in sklearn 

lr = LinearRegression()
lr.fit(x_train_reshaped, y_train_reshaped)

pred_slope = lr.coef_
pred_b = lr.intercept_
y_pred= lr.predict(x_test_reshaped)
residuals = y_test - y_pred.reshape(y_pred.shape[0],)

# # fitting the model line to the data 
# model_line = (pred_slope*x)+pred_b
# model_line_reshaped = model_line.reshape(model_line.shape[1])
# plt.figure()
# plt.plot(x, model_line_reshaped, color="red")
# plt.scatter(x, y_noised)


# # get resiuduals and plot it 
# y_pred= lr.predict(x_test_reshaped)

# residuals = y_test - y_pred.reshape(y_pred.shape[0],)
# plt.figure()
# plt.scatter(x_test,residuals)
# plt.show()


Figure 3 is a plot of residuals. A residual is the difference between the actual value, which are the blue point in Figure 2 and the predicted values, which fall on the red line. One of the assumptions of linear regression is that the residuals are drawn from a normal distribution, another way of saying this would be that we if plot a histogram of it, we expect it but normal-like. Take a look at figure 4 below. The histogram has a single peak around 0 and falls off on either side. However can we  call this a representation of a normal distribution? It is not clear. 

The histogram is one graphical way to say that the data comes from a normal distribution, but the histogram can be deceptive since changing the number of bins alter the shape of the distribution and this may lead to some confusion. 




In [3]:

residual_df = pd.DataFrame(data=residuals,columns=["residuals"])

# callback function for the slider
def change_bins(number_bins):
    return px.histogram(residual_df, x="residuals", nbins=int(number_bins))

slider_obj =  widgets.FloatSlider(value=20, min=10, max=100,step=5, description="Num of bins", continuous_update=False)
interact(change_bins, number_bins=slider_obj);

interactive(children=(FloatSlider(value=20.0, continuous_update=False, description='Num of bins', min=10.0, st…


Take a look at figure 4 where you can change the number of bins in the histogram. The shape of the histogram remains the same for smaller bin sizes (larger number of bins) but as you go to larger bin sizes, there is variation in the shape of the histogram. This is especially a problem with small datasets. With large datasets, the variations will in the shape will be small. 

An alternative to using histogram is to use something called a quantile quantile plot. A quantile quantile plot is a scatter plot where we plot the data set values vs normal distribution values for quantiles determined from the dataset. The way it works is that we first get the quantile value for each of the data points in our dataset. Once we have these values, we calculate the independent variable values on the normal distribution. Each of these quantile values are essentially the percentage of the area under the curve. So for example the 0.10 quantile would be 10% of the area under a normal curve. The x coordinate values of the normal distribution for these quantile values become the x values for the quantile quantile plot. 

In python there is an easy way of doing this. First we can get data for the quantile quantile plot from below.

To generate the data for the quantile quantile plot we must define the quantiles for which we need the values from the normal distribution, so first we write- 
```python 
num_divisions = residuals.shape[0]+1
quantiles = np.arange(1,residuals.shape[0])/num_divisions
```

The *num_divisions* tells us the how many divisions in the normal distribution we need. This if you run the code above you should see the quantiles will be - 

```[0.00497512, 0.00995025, 0.01492537, 0.0199005 ...]```

So if we have *n* data points in a dataset, we need to divide the normal distribution into *n+1* regions to get *n* from the normal distribution. 

So for example in our case we have 200 points hence we need to have 201 divisions in the normal distribution. Once we have the quantile values we then insert them into the *ppf* function. This is the inverse of the cumulative distribution which gives us the z-score of the normal distribution for a given quantile value. 

The quantile values that we are feeding into the *ppf* function are nothing more than the ratio of the area under the normal curve normalized by 1. 

```python 
qq_x_data = sps.norm.ppf(quantiles)
``` 
These values are the x values for the qq plot, we get the y values by just sorting the residuals

```python 
qq_y_data = np.sort(residuals)
```

Next we need to get the data for plotting the guide line. For that we need two points so we can determine the slope and y intercept of the line. For this, we are going adopt the recommendation of Cleveland in *Visualizing data* [[1](https://dl.acm.org/citation.cfm?id=529269)]. Do not fit a regression line to the points on the qq plot will not satisfy the assumptions of linear regression (think about it! why?)

The x values for the first and third quartiles will come from normal distribution. The y values will come from our ordered dataset. Hence we write- 

```python

# guide line data
line_x0 = sps.norm.ppf(0.25)
line_x1 = sps.norm.ppf(0.75)

line_y0 = np.quantile(residuals, 0.25)
line_y1 = np.quantile(residuals, 0.75)


```

using these two points we form a line-
```python 


slope = (line_y1-line_y0)/(line_x1-line_x0)
line_intercept = line_y1 - (slope*line_x1)

x_range_line = np.arange(-3,3,0.001)
y_values_line = (slope*x_range_line) + line_intercept 

```
In the below cell the full code for the line is shown.


In [4]:
num_divisions = residuals.shape[0]+1
quantiles = np.arange(1,residuals.shape[0])/num_divisions

# scatter data 
qq_x_data = sps.norm.ppf(quantiles)
qq_y_data = np.sort(residuals)

# guide line data
line_x0 = sps.norm.ppf(0.25)
line_x1 = sps.norm.ppf(0.75)

line_y0 = np.quantile(residuals, 0.25)
line_y1 = np.quantile(residuals, 0.75)


slope = (line_y1-line_y0)/(line_x1-line_x0)
line_intercept = line_y1 - (slope*line_x1)

x_range_line = np.arange(-3,3,0.001)
y_values_line = (slope*x_range_line) + line_intercept  


Now that we have the data for the qq plot and the reference line we will use plotly to plot them. The code for it is in the next cell. 


In [5]:

fig = go.Figure()

fig.add_trace(go.Scatter(x=qq_x_data, y=qq_y_data, mode='markers', marker={'color':'blue'}, name="qq data"))
fig.add_trace(go.Scatter(x=x_range_line, y=y_values_line, mode='lines', marker={'color':'red'}, name="guide line"))


fig['layout'].update(title='Quantile-Quantile plot',
                     xaxis={
                            'title': 'Theoritical Quantities',
                            'zeroline': True
                           },
                    yaxis={
                            'title': 'Sample Quantities'
                          },
                   showlegend=True,
                   )


iplot(fig)

The way to read a qq plot is to see how many points fall along the reference line. If majority of lines fall onto this line then it can be assumed that the data follows a normal distribution. If majority of plots do not follow this trend then we can say that the data does not have a normal trend. The qq plot is a simple graphical way to decipher if data set follows a normal distribution or not. 

Suppose we have a dataset that follows a uniform distribution. then the qq plot will look different. 



In [6]:
uniform_data = np.random.uniform(0,10,1000)

num_divisions = uniform_data.shape[0]+1
quantiles = np.arange(1,uniform_data.shape[0])/num_divisions

# scatter data 
qq_uniform_x = sps.norm.ppf(quantiles)
qq_uniform_y = np.sort(uniform_data)

line_y0 = np.quantile(uniform_data, 0.25)
line_y1 = np.quantile(uniform_data, 0.75)


slope = (line_y1-line_y0)/(line_x1-line_x0)
line_intercept = line_y1 - (slope*line_x1)

# points to plot the line 
x_uniform = np.arange(-3,3,0.001)
y_uniform = (slope*x_range_line) + line_intercept  



fig = go.Figure()

fig.add_trace(go.Scatter(x=qq_uniform_x, y=qq_uniform_y, mode='markers', marker={'color':'blue'}, name="qq data"))
fig.add_trace(go.Scatter(x=x_uniform, y=y_uniform, mode='lines', marker={'color':'red'}, name="guide line"))


fig['layout'].update(title='Quantile-Quantile plot',
                     xaxis={
                            'title': 'Theoritical Quantities',
                            'zeroline': True
                           },
                    yaxis={
                            'title': 'Sample Quantities'
                          },
                   showlegend=True,
                   )


iplot(fig)

In fact we can do this for many other distribution types. Here is a plotly plot comparing the residuals data to multiple distributions



In [7]:
def get_qqdata(observed_data):
    np.random.seed(0)
    num_divisions = observed_data.shape[0]+1
    quantiles = np.arange(1,observed_data.shape[0])/num_divisions

    # scatter data 
    qq_x = sps.norm.ppf(quantiles)
    qq_y = np.sort(observed_data)

    line_y0 = np.quantile(observed_data, 0.25)
    line_y1 = np.quantile(observed_data, 0.75)


    slope = (line_y1-line_y0)/(line_x1-line_x0)
    line_intercept = line_y1 - (slope*line_x1)

    # points to plot the line 
    x_line = np.arange(-3,3,0.001)
    y_line = (slope*x_range_line) + line_intercept  

    qq_data = {'qqx': qq_x, 'qqy':qq_y, 'linex': x_line, 'liney':y_line}
    
    return qq_data


def dist_qqplot(dist_data, dist_name) :


    qq_uniform = get_qqdata(dist_data)



    fig = make_subplots(rows=1, cols=2,subplot_titles=["data histogram", "qq plot"])

    fig.add_trace(go.Histogram(x=dist_data, name=dist_name), row=1, col=1)
    fig.add_trace(go.Scatter(x=qq_uniform["qqx"] , y=qq_uniform["qqy"], mode='markers', marker={'color':'blue'}, name="qq data"), row=1,col=2)
    fig.add_trace(go.Scatter(x=qq_uniform["linex"], y=qq_uniform["liney"], mode='lines', marker={'color':'red'}, name="guide line"), row=1,col=2)

    fig.update_xaxes(title_text="data values" ,row=1, col=1)
    fig.update_xaxes(title_text="theoretical quantiles", range=[-4,4], row=1, col=2)
    fig.update_yaxes(title_text="Count", row=1, col=1)
    fig.update_yaxes(title_text="observed quantiles" , row=1, col=2)
    
    return iplot(fig)



def distributions(dist): 
    
    # change parameter values here to see how the qq plot can change
    selected_data ={"triangular": np.random.triangular(10,100,115,1000),
                    'lognormal': np.random.lognormal(10,1,1000),
                    'chi square': np.random.chisquare(8,1000)}
    
    return dist_qqplot(selected_data[dist], dist )

toggle_obj = widgets.ToggleButtons(description="data distributions", options=["triangular","lognormal", "chi square"])
interact(distributions, dist=toggle_obj)

interactive(children=(ToggleButtons(description='data distributions', options=('triangular', 'lognormal', 'chi…

<function __main__.distributions(dist)>

In [8]:
np.random.log

AttributeError: module 'numpy.random' has no attribute 'log'

In [ ]:
quantiles[4]

In [ ]:
from scipy.stats import norm

x_axis = np.zeros(residuals.shape[0])
y = np.sort(residuals)
x_norm = np.arange(-5, 5, 0.001)
y_norm = norm.pdf(x_norm,0,1)

fig = make_subplots(rows=1, cols=2)

fig.add_trace(go.Scatter({'x':x_axis, 'y':y, 'mode':'markers'}), row=1, col=1)
fig.add_trace(go.Scatter({'x':x_norm, 'y':y_norm}), row=1, col=2) 
fig.update_layout(shapes=[go.layout.Shape(type='line', yref="y2", #Also tried with y and paper))
    x0=0,
    y0=0,
    x1=0,
    y1=2,
    line=dict(
        color="Red",
        width=3
        )

               )])


In [ ]:
sps.norm.ppf(0.50)

In [ ]:
np.quantile(residuals,0.5)

In [ ]:
np.median(residuals)